## Import

In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

## Data Load

In [2]:
data = pd.read_csv('./train.csv')

# person_id 컬럼 제거
X_train = data.drop(['person_id', 'login'], axis=1)
y_train = data['login']

## Hyperparameters Search

In [3]:


# # GridSearchCV를 위한 하이퍼파라미터 설정
# param_search_space = {
#     'n_estimators': [10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
#     'criterion': ['gini', 'entropy'],
#     'max_depth': [None, 3, 5, 7, 10, 15, 20, 30],
#     'min_samples_split': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#     'min_samples_leaf': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
#     'min_weight_fraction_leaf': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_leaf_nodes': [None, 3, 5, 7, 10, 15],
#     'min_impurity_decrease': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#     'bootstrap': [True, False]
# }

# # RandomForestClassifier 객체 생성
# rf = RandomForestClassifier(random_state=42)

# # GridSearchCV 객체 생성
# grid_search = GridSearchCV(estimator=rf, param_grid=param_search_space, cv=3, n_jobs=-1, verbose=2, scoring='roc_auc')

# # GridSearchCV를 사용한 학습
# grid_search.fit(X_train, y_train)

# # 최적의 파라미터와 최고 점수 출력
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# best_params, best_score

In [4]:
# n_estimators,criterion,max_depth,min_samples_split,min_samples_leaf,min_weight_fraction_leaf,max_features,max_leaf_nodes,min_impurity_decrease,bootstrap
# n_estimators : 10 ~ 1000 사이의 양의 정수
# criterion :  'gini', 'entropy'. 'gini'는 진니 불순도를, 'entropy'는 정보 이득
# max_depth : None 또는 양의 정수
# min_samples_split : 2 이상의 정수 또는 0과 1 사이의 실수
# min_samples_leaf : 1 이상의 정수 또는 0과 0.5 사이의 실수
# min_weight_fraction_leaf :  0.0에서 0.5 사이의 실수
# max_features :  'auto', 'sqrt', 'log2' 또는 양의 정수/실수
# max_leaf_nodes :  None 또는 양의 정수
# min_impurity_decrease : 0.0 이상의 실수
# bootstrap : True, False

In [5]:
import optuna
from sklearn.pipeline import make_pipeline

# Valid parameters are: ['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 
#                        'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 
#                        'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].

def objective(trial):
    params = {
        "n_estimators": [trial.suggest_int("n_estimators",10,1000)],
        "criterion": [trial.suggest_categorical("criterion", ["gini", "entropy"])],
        "max_depth": [trial.suggest_categorical("max_depth", [None] + list(range(4, 31)))],
        "min_samples_split": [trial.suggest_int("min_samples_split", 2, 50)],
        "min_samples_leaf": [trial.suggest_int("min_samples_leaf", 1, 50)],
        "min_weight_fraction_leaf": [trial.suggest_float("min_weight_fraction_leaf", 0.0, 0.5)],
        "max_features": [trial.suggest_categorical("max_features", [None, "sqrt", "log2"])],
        "max_leaf_nodes": [trial.suggest_categorical("max_leaf_nodes", [None] + list(range(2, 101)))],
        "min_impurity_decrease": [trial.suggest_float("min_impurity_decrease", 0.0, 1.0)],
        "bootstrap": [trial.suggest_categorical("bootstrap", [True, False])],
        "verbose": [0],
    }
    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(estimator=rf, cv=3, n_jobs=-1, param_grid=params, scoring='roc_auc')
    grid_search.fit(X_train, y_train)
    # 최적의 파라미터와 최고 점수 출력
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    return best_score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1000)

best_params = study.best_params

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-03-25 12:16:27,844] A new study created in memory with name: no-name-b54528a3-28b4-43d4-8988-7a18f92a67da


[I 2024-03-25 12:16:29,660] Trial 0 finished with value: 0.5 and parameters: {'n_estimators': 260, 'criterion': 'gini', 'max_depth': 9, 'min_samples_split': 27, 'min_samples_leaf': 50, 'min_weight_fraction_leaf': 0.2957894202269661, 'max_features': 'log2', 'max_leaf_nodes': 19, 'min_impurity_decrease': 0.3091563908686653, 'bootstrap': False}. Best is trial 0 with value: 0.5.
[I 2024-03-25 12:16:32,413] Trial 1 finished with value: 0.5 and parameters: {'n_estimators': 772, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_split': 34, 'min_samples_leaf': 7, 'min_weight_fraction_leaf': 0.26757269795475674, 'max_features': None, 'max_leaf_nodes': 54, 'min_impurity_decrease': 0.9993970650206575, 'bootstrap': False}. Best is trial 0 with value: 0.5.
[I 2024-03-25 12:16:33,962] Trial 2 finished with value: 0.5 and parameters: {'n_estimators': 265, 'criterion': 'entropy', 'max_depth': 21, 'min_samples_split': 48, 'min_samples_leaf': 50, 'min_weight_fraction_leaf': 0.33868244447851625, 'max_

In [6]:
study.best_value

0.7876455368502926

## Submission

In [7]:
submit = pd.read_csv('./sample_submission.csv')

# 찾은 최적의 파라미터들을 제출 양식에 맞게 제출
for param, value in best_params.items():
    if param in submit.columns:
        submit[param] = value
        print("%s : %s" %(param,value))

submit.to_csv('./baseline_submit.csv', index=False)

n_estimators : 230
criterion : gini
max_depth : 18
min_samples_split : 38
min_samples_leaf : 20
min_weight_fraction_leaf : 0.04314084129884324
max_features : log2
max_leaf_nodes : 66
min_impurity_decrease : 0.00060404316576883
bootstrap : False
